# Requirements 📋✅

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
# import cv2

# Needed for the data
from sklearn.model_selection import train_test_split
from PIL import Image

from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import convert_to_tensor
from tensorflow.image import rgb_to_grayscale
from tensorflow.keras.models import load_model as tfk__load_model
import tensorflow as tf


2023-12-06 15:39:24.214380: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-06 15:39:24.859652: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-12-06 15:39:24.859670: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-12-06 15:39:24.936769: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-06 15:39:26.850104: W tensorflow/stream_executor/platform/de

In [2]:
from tensorflow.keras.callbacks import EarlyStopping


In [3]:
from tensorflow import convert_to_tensor


# Hopefully fixing cuda

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [5]:
!echo $CUDA_VISIBLE_DEVICES


-1


# The Dataset location 📍

## Label Data Load ⏳

In [8]:
# params
batch_size = 32
img_height = 256
img_width = 256
validation_split = 0.2
num_classes = 4


In [8]:
train_dataset = image_dataset_from_directory(
    directory='/home/kyrill/code/pt-ai/pt-ai/raw_data/processed_data_03',
    labels='inferred',
    label_mode='categorical',
    color_mode='grayscale',
    validation_split=validation_split,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)


Found 4772 files belonging to 4 classes.
Using 3818 files for training.


2023-12-01 14:09:02.421058: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [9]:
validation_dataset = image_dataset_from_directory(
    directory='/home/kyrill/code/pt-ai/pt-ai/raw_data/processed_data_03',
    labels='inferred',
    label_mode='categorical',
    color_mode='grayscale',
    validation_split=validation_split,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)


Found 4772 files belonging to 4 classes.
Using 954 files for validation.


In [10]:
type(train_dataset)


tensorflow.python.data.ops.prefetch_op._PrefetchDataset

## Preprocessing ⚙️

### CNN Model Function 🦾💻🧠

In [11]:
def initialize_CNN():
    model = models.Sequential()

    # Preprocessing layers
    model.add(layers.CenterCrop(height=350, width=450, input_shape=[256,256,1]))

    # Build of the Model
    model.add(layers.Conv2D(filters=8, kernel_size=(4,4), activation='relu', padding='same'))
    model.add(layers.MaxPool2D((2,2)))
    model.add(layers.Conv2D(16,(3,3), activation='relu'))
    model.add(layers.MaxPool2D((2,2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(4, activation='softmax'))

    # Compilation of the Model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [12]:
# Summary of the CNN model
tmp_first_CNN = initialize_CNN()
tmp_first_CNN.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 center_crop (CenterCrop)    (None, 350, 450, 1)       0         
                                                                 
 conv2d (Conv2D)             (None, 350, 450, 8)       136       
                                                                 
 max_pooling2d (MaxPooling2  (None, 175, 225, 8)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 173, 223, 16)      1168      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 86, 111, 16)       0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 152736)            0

In [15]:
es = EarlyStopping(patience=1, restore_best_weights=True)


In [17]:
history = tmp_first_CNN.fit(train_dataset,
                            epochs=1,
                            validation_data=validation_dataset,
                            batch_size=16,
                            callbacks=[es])


120/120 [==============================] - 195s 2s/step - loss: 1.3670 - accuracy: 0.3279 - val_loss: 1.3507 - val_accuracy: 0.3532


In [18]:
tmp_first_CNN.evaluate(validation_dataset)


30/30 [==============================] - 9s 285ms/step - loss: 1.3507 - accuracy: 0.3532


[1.350656270980835, 0.3532494902610779]

In [6]:
from PIL import Image


In [99]:
def preprocessing_images(image_path):
    pred_image = Image.open(image_path)
    X_pred = np.array(pred_image)
    X_pred = tf.convert_to_tensor(X_pred, dtype=tf.float32)
    X_pred = tf.image.rot90(X_pred, k=-1)

    X_pred = X_pred[:,:,:3]/255.
    X_pred = tf.image.resize_with_pad(X_pred, 256,256)
    X_pred = rgb_to_grayscale(X_pred)

    X_pred = tf.expand_dims(X_pred, axis=0)

    # plt.imshow(rgb_to_grayscale(X_pred[:,:,:3]/255.), cmap='gray')
    return X_pred


In [128]:
preprocessing_images('../raw_data/test_images/IMG_8806.jpg')


<tf.Tensor: shape=(1, 256, 256, 1), dtype=float32, numpy=
array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]]], dtype=float32)>

In [118]:
def prediction_model(model, image_preprocessed):
    """Try and predict an image from the Dataset"""

    y_pred = model.predict(image_preprocessed)
    print(f'✅ Prediction complete. Pose: {y_pred}')
    return y_pred


In [127]:
fitted_model = tfk__load_model('../raw_data/models/model.h5')
res1 = prediction_model(fitted_model, preprocessing_images('../raw_data/test_images/IMG_8803 (1).jpg'))
res2 = prediction_model(fitted_model, preprocessing_images('../raw_data/test_images/IMG_8804.jpg'))
res3 = prediction_model(fitted_model, preprocessing_images('../raw_data/test_images/IMG_8806.jpg'))

print(res1, res2, res3)


1/1 [==============================] - 0s 254ms/step
✅ Prediction complete. Pose: [[0.23731439 0.09497999 0.5056726  0.1620331 ]]
1/1 [==============================] - 0s 36ms/step
✅ Prediction complete. Pose: [[0.23759201 0.09759529 0.49646252 0.16835018]]
1/1 [==============================] - 0s 23ms/step
✅ Prediction complete. Pose: [[0.23676254 0.09712522 0.49954796 0.16656429]]
[[0.23731439 0.09497999 0.5056726  0.1620331 ]] [[0.23759201 0.09759529 0.49646252 0.16835018]] [[0.23676254 0.09712522 0.49954796 0.16656429]]


In [3]:
import requests


In [ ]:

# URL of your FastAPI endpoint
url = 'http://0.0.0.0:8000/uploadfile/'

# Path to the local image file
file_path = "/home/kyrill/code/pt-ai/pt-ai/raw_data/test_images/IMG_8803 (1).jpg"

# Open the file in binary mode
with open(file_path, 'rb') as f:
    # Define the file as a dictionary. The key ('file' in this case)
    # should match the name of the parameter in your FastAPI endpoint
    files = {'file': (file_path, f, 'image/jpeg')}

    # Make the POST request
    response = requests.post(url, files=files)

# Print the response from the server
print(response.text)


In [139]:
# URL of your FastAPI endpoint
url = 'http://0.0.0.0:8000/skeletonizer/'

# Path to the local image file
file_path = "/home/kyrill/code/pt-ai/pt-ai/raw_data/test_images/IMG_8806.jpg"

# Open the file in binary mode
with open(file_path, 'rb') as f:
    # Define the file as a dictionary. The key ('file' in this case)
    # should match the name of the parameter in your FastAPI endpoint
    files = {'file': (file_path, f, 'image/jpeg')}

    # Make the POST request
    response = requests.post(url, files=files)

# Print the response from the server
print(response.text)


{"keypoints":"{\"left_elbow_a\":{\"0\":125.1089438573},\"right_elbow_a\":{\"0\":36.4066055782},\"left_shoulder_b\":{\"0\":102.6167347695},\"left_shoulder_a\":{\"0\":97.4974123639},\"right_shoulder_b\":{\"0\":164.3084513532},\"right_shoulder_a\":{\"0\":57.5133483589},\"left_hip_a\":{\"0\":87.7763387975},\"left_hip_b\":{\"0\":118.3856644507},\"left_hip_c\":{\"0\":153.838003582},\"right_hip_a\":{\"0\":117.2128936494},\"right_hip_b\":{\"0\":15.3016454768},\"right_hip_c\":{\"0\":101.9112481726},\"left_knee_a\":{\"0\":114.8381971571},\"right_knee_a\":{\"0\":79.3844610247}}"}


In [140]:
dict_var = eval(response.text)

type(dict_var)


dict

In [141]:
url = 'http://0.0.0.0:8000/automl_model/'

# Path to the local image file
input_for_model = {"data":dict_var}
response = requests.post(url, json=input_for_model)

# Print the response from the server
print(response.text)


{"predict":"[2]"}


In [108]:
type(response.text)


str